

---


# **Twitter Data Collection**
#### ***Look before you Leap : Leveraging Predictive Models to Improve Automotive Safety and Travel Time***



---



**Final Project**
CSCI 35300/79502   |   Fall 2020 

##### Hannah Do








#### Used Snscrape and Tweepy to collect queried tweets :

In [1]:
! pip3 install snscrape

In [2]:
! pip install tweepy

In [13]:
# Storing twitter URLs in according folders

## TotalTrafficNYC data inconsistent  - sparse at the beginning of the year
## NYPD traffic and NY_DOT - too much irrelevant information / news or ad centered tweets

! snscrape twitter-search "@511NY since:2020-06-01 until:2020-06-15" > data_twitter/511_6a.txt
! snscrape twitter-search "@511NY since:2020-06-16 until:2020-07-01" > data_twitter/511_6b.txt

! snscrape twitter-search "@511NY since:2020-07-02 until:2020-07-15" > data_twitter/511_7a.txt
! snscrape twitter-search "@511NY since:2020-07-16 until:2020-08-01" > data_twitter/511_7b.txt

! snscrape twitter-search "@511NY since:2020-08-02 until:2020-08-15" > data_twitter/511_8a.txt
! snscrape twitter-search "@511NY since:2020-08-16 until:2020-09-01" > data_twitter/511_9b.txt

! snscrape twitter-search "@511NY since:2020-09-02 until:2020-09-15" > data_twitter/511_9a.txt
! snscrape twitter-search "@511NY since:2020-09-16 until:2020-10-01" > data_twitter/511_9b.txt

! snscrape twitter-search "@511NY since:2020-10-02 until:2020-10-15" > data_twitter/511_10a.txt
! snscrape twitter-search "@511NY since:2020-10-16 until:2020-11-01" > data_twitter/511_10b.txt

! snscrape twitter-search "@511NY since:2020-11-02 until:2020-11-15" > data_twitter/511_11a.txt
! snscrape twitter-search "@511NY since:2020-11-16 until:2020-12-01" > data_twitter/511_11b.txt



In [4]:

# Import all the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

import datetime, string, re
import sys

import tweepy
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from collections import Counter

print('All Libraries Sucessfully Imported')


All Libraries Sucessfully Imported


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/doguma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/doguma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/doguma/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


---
#### Twitter Developer Accounts required for API keys :

In [6]:
# # Hannah
# consumer_key = 'bOgKgIHGuCybRIOFxdxTuFWM3'
# consumer_secret = 'XcP6KypHOQurNQvySOVtqy4Fz6TyvOHNmFwAmMmF54AKTG88Ui'
# access_token = '1317375977203265537-305NGnHrsADD2EqTqbZGRU9ADAmMFx'
# access_token_secret = 'dypXTOkWumb4mC8mtZkrHN4s2q8agkuh8BORi41zLBhk8'


# # Andrew
# consumer_key = 'Tin2pOJsulrEFcQMTpcnSlrTc'
# consumer_secret = '58TBGb2n5cp0ct5VocwwJDNTSTMBGVyJPsUkIEOkKbZySmMhHQ'
# access_token = '1265794282436333569-RC1dJpKAmhmKK8t2wCCSmPl04KY3ll'
# access_token_secret = 'djW6cDWDWjCDakqteYvsLI36eFSn82GsRN6aeOEQbLpWx'


# Phillip
consumer_key = '9WjJiKWJKoeoy6P0r4wixXKj1'
consumer_secret = 'whwOu0J3gIsjexUifZfmrgFNDjJMPGepJkYUPScE5aYKU2IZjX'
access_token = '1316846398197899264-PDFrLnEDyoD9jtjriKnQ93FEg4MA2X'
access_token_secret = '9JGc9dAZP4lP78xFQfHNrpd4K7ogG49n6mxUqMtyuaTVh'



auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


sid = SentimentIntensityAnalyzer()
# initializing Vader Sentiment Analyzer


In [55]:
def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at}
#             tweet_elem = {
#                      "tweet":status.full_text,
#                 "date":status.created_at
#                         }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    return empty_data

- Timeline segmented to smaller units due to long processing hours :

In [60]:

# '511_6a', '511_6b', '511_7a', '511_7b', '511_8a', '511_8b', '511_9a', '511_9b', '511_10a', '511_10b', '511_11a'
t_files = ['511_11b']


-  txt files (link of the tweets) converted to csv files (content of the tweets) :

In [61]:

for t in t_files:

    tweet_url = pd.read_csv("data_twitter/" + t +".txt", index_col= None, header = None, names = ["links"])
    tweet_url.head()

    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)

    ids = tweet_url['id'].tolist()

    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1


    for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        temp2 = pd.DataFrame(fetch_tw(batch))
        temp2.to_csv("data_twitter/" + t +".csv", mode="a")


- each csv file read as dataframe :

In [62]:

tweet_text = pd.read_csv("data_twitter/511_6a.csv", index_col= None, header = None)
tweet_text

temp = pd.DataFrame()
temp['tweet'] = tweet_text[3]

temp.drop(temp.head(1).index,inplace=True)

temp['score_neutral'] = temp['tweet'].apply(lambda x: list(sid.polarity_scores(str(x)).values())[1])
# get neutral sentiment of the tweet

temp['score_polar'] = temp['tweet'].apply(lambda x: list(sid.polarity_scores(str(x)).values())[3])
# get compound score - overall polar sentiment of the tweet

temp


,tweet,score_neutral,score_polar
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000
...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000


- text processing on tweet texts

In [65]:
# separate camel cases and remove punctuations
# not needed for geocoding

def separate_camelcase(text):
    text = re.sub(r'\'s', '',text)
    text = re.sub(r'\'', '',text)
    text = re.sub(r'[^\w\s]',' ',text)
    text = re.sub(r"([a-z])([A-Z]+)", r"\1 \2", str(text))

    return text


# Replace acronyms and remove redundant words

def replace_acronyms(text):
    # text  = "".join([char for char in text if char not in excluded_words])  
    text = re.sub('i76', ' interstate 76 ', text)
    text = re.sub('i84', ' interstate 84 ', text)
    text = re.sub('i87', ' interstate 87 ', text)
    text = re.sub('i91', ' interstate 91 ', text)
    text = re.sub('i95', ' interstate 95 ', text)
    text = re.sub('i190', ' interstate 190 ', text)
    text = re.sub('i295', ' interstate 295 ', text)
    
    text = re.sub('eb', ' east bound ', text)
    text = re.sub('sb', ' south bound ', text)
    text = re.sub('nb', ' north bound ', text)
    text = re.sub('wb', ' west bound ', text)

    text = re.sub('queens', ' queens ', text)
    text = re.sub('nj', ' new jersey ', text)
    text = re.sub('nyc', ' new york city ', text)
    text = re.sub('brooklyn', ' brooklyn ', text)
    text = re.sub('new york', ' new york ', text)        
    text = re.sub('staten island', ' staten island ', text)
    text = re.sub('newyork', ' new york ', text)
    text = re.sub('nypd', '', text)
    text = re.sub('ny', ' new york ', text)
    text = re.sub('bronx', ' bronx ', text)
#     text = re.sub('accident', '', text)
#     text = re.sub('parks', '', text)
#     text = re.sub('sirens', '', text)
#     text = re.sub('incident', '', text)
    text = re.sub('rte', ' route ', text)
    text = re.sub('rt', ' route ', text)
    text = re.sub('bklyn', ' brooklyn ', text)
    text = re.sub('bk', ' brooklyn ', text)
    text = re.sub('way', ' way ', text)
    text = re.sub('expy', ' expressway ', text)
#     text = re.sub('crash', '', text)
    text = re.sub('aka', '', text)
    text = re.sub('bqe', ' brooklyn queens expressway ', text)
    text = re.sub('511ny', '', text)

    return text 

temp['separate camel cases'] = temp['tweet'].apply(lambda x: separate_camelcase(x))

temp['replace acronyms'] = temp['separate camel cases'].apply(lambda x: x.lower())

temp['replace acronyms'] = temp['replace acronyms'].apply(lambda x: replace_acronyms(x))

temp

,tweet,score_neutral,score_polar,separate camel cases,replace acronyms
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,Cleared Incident on Southern State Parkway W...,cleared incident on southern state park way ...
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000,Updated Incident on River Line Both directio...,updated incident on river line both directio...
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027,Cleared Incident on NJ139Lower EB at New Jer...,cleared incident on new jersey 139lower ea...
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,Construction on I84E EB at Exit 39A RTE 9 SB...,construction on interstate 84 e east bound ...
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,Incident on NJ181 Both directions at CR 517 S...,incident on new jersey 181 both directions a...
...,...,...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000,Construction on NJ3 Both directions from Park...,construction on new jersey 3 both directions...
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027,Cleared Incident on M14DBus at 14th Street,cleared incident on m14dbus at 14th street
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960,Construction on NY9A Both directions from Low...,construction on new york 9a both directions ...
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000,Construction on Q12Bus at Alameda Ave https ...,construction on q12bus at alameda ave https ...


In [64]:
stopword = nltk.corpus.stopwords.words('english')

In [66]:

# Tokenize the text
def tokenization(text):
    text = re.split('\W+', text)
    return text

temp['remove stopwords'] = temp['replace acronyms'].apply(lambda x: tokenization(x))


# Remove stopwords

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
temp['remove stopwords'] = temp['remove stopwords'].apply(lambda x: remove_stopwords(x))

temp.head(10)

# Lemmatization and Stemming skipped due to losing of necessary word forms

# Convert the tokens back to string

def listToString(s):  
    str1 = ""  
      
    for ele in s:  
        str1 += ele + " "  
      
    return str1

temp['remove stopwords'] = temp['remove stopwords'].apply(lambda x: listToString(x))
temp


,tweet,score_neutral,score_polar,separate camel cases,replace acronyms,remove stopwords
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,Cleared Incident on Southern State Parkway W...,cleared incident on southern state park way ...,cleared incident southern state park way west ...
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000,Updated Incident on River Line Both directio...,updated incident on river line both directio...,updated incident river line directions trenton...
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027,Cleared Incident on NJ139Lower EB at New Jer...,cleared incident on new jersey 139lower ea...,cleared incident new jersey 139lower east boun...
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,Construction on I84E EB at Exit 39A RTE 9 SB...,construction on interstate 84 e east bound ...,construction interstate 84 e east bound exit 3...
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,Incident on NJ181 Both directions at CR 517 S...,incident on new jersey 181 both directions a...,incident new jersey 181 directions cr 517 spa ...
...,...,...,...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000,Construction on NJ3 Both directions from Park...,construction on new jersey 3 both directions...,construction new jersey 3 directions park ave ...
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027,Cleared Incident on M14DBus at 14th Street,cleared incident on m14dbus at 14th street,cleared incident m14dbus 14th street
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960,Construction on NY9A Both directions from Low...,construction on new york 9a both directions ...,construction new york 9a directions lower stre...
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000,Construction on Q12Bus at Alameda Ave https ...,construction on q12bus at alameda ave https ...,construction q12bus alameda ave https co dov n...


#### Geocoding on each text through ArcGIS : returns location (longitude, latitude) of the text
- takes at least 5 hours to process

In [67]:

from arcgis.gis import GIS
from arcgis.geocoding import geocode
my_gis = GIS()

temp['latitude'] = 0
temp['longitude'] = 0


In [68]:

temp2 = pd.DataFrame()
temp2 = temp


In [72]:

for i in range(len(temp2['remove stopwords'])):
    string_temp = str(temp2['remove stopwords'][i+1])
    string_temp = string_temp + ' new york'
    
    if len(string_temp)<100:
        locat = geocode(string_temp)
        temp2['longitude'][i+1] = locat[0]['location']['x']
        temp2['latitude'][i+1] = locat[0]['location']['y']
    else:
        temp2['latitude'][i+1] = 0
        temp2['longitude'][i+1] = 0
        

<ipython-input-72-da36a05430e3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['longitude'][i+1] = locat[0]['location']['x']
<ipython-input-72-da36a05430e3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['latitude'][i+1] = locat[0]['location']['y']
<ipython-input-72-da36a05430e3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['latitude'][i+1] = 0
<ipython-input-72-da36a05430e3>:11: SettingWithCopyWarning: 
A va

RequestException: A general expection was raised: HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=None)

In [73]:
temp2

,tweet,score_neutral,score_polar,separate camel cases,replace acronyms,remove stopwords,latitude,longitude
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,Cleared Incident on Southern State Parkway W...,cleared incident on southern state park way ...,cleared incident southern state park way west ...,40.7924,-73.8278
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000,Updated Incident on River Line Both directio...,updated incident on river line both directio...,updated incident river line directions trenton...,0,0
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027,Cleared Incident on NJ139Lower EB at New Jer...,cleared incident on new jersey 139lower ea...,cleared incident new jersey 139lower east boun...,40.7031,-74.1532
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,Construction on I84E EB at Exit 39A RTE 9 SB...,construction on interstate 84 e east bound ...,construction interstate 84 e east bound exit 3...,0,0
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,Incident on NJ181 Both directions at CR 517 S...,incident on new jersey 181 both directions a...,incident new jersey 181 directions cr 517 spa ...,40.7146,-74.0071
...,...,...,...,...,...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000,Construction on NJ3 Both directions from Park...,construction on new jersey 3 both directions...,construction new jersey 3 directions park ave ...,,
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027,Cleared Incident on M14DBus at 14th Street,cleared incident on m14dbus at 14th street,cleared incident m14dbus 14th street,,
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960,Construction on NY9A Both directions from Low...,construction on new york 9a both directions ...,construction new york 9a directions lower stre...,,
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000,Construction on Q12Bus at Alameda Ave https ...,construction on q12bus at alameda ave https ...,construction q12bus alameda ave https co dov n...,,


In [74]:
temp2['longitude'] = temp2['longitude'].apply(lambda x: float(x))
temp2['latitude'] = temp2['latitude'].apply(lambda x: float(x))

temp2 = temp2[temp2['latitude'] > 40.5]
temp2 = temp2[temp2['latitude'] < 41]
temp2 = temp2[temp2['longitude'] < -73]
temp2 = temp2[temp2['longitude'] > -75]

temp2

ValueError: could not convert string to float: ''

In [23]:
temp2 = temp2[temp2['latitude'] !=0]
temp2.drop(columns = ['separate camel cases', 'replace acronyms'], inplace=True)
temp2

,tweet,score_neutral,score_polar,remove stopwords,latitude,longitude
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,cleared incident southern state park way wb ex...,40.7924,-73.8278
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027,cleared incident new jersey 139lower eb new je...,44.2215,-79.4321
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,construction i84e eb exit 39a e 9 sb https co ...,40.7146,-74.0071
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,incident new jersey 181 directions cr 517 spaa...,40.7146,-74.0071
6,Incident on #Q15ABus from Roosevelt Ave: Main ...,1.000,0.0000,incident q15abus roosevelt ave main st rooseve...,40.6786,-73.589
...,...,...,...,...,...,...
27082,Cleared: Incident on #M14ABus at 14th Street.,0.811,0.1027,cleared incident m14abus 14th street,40.6701,-73.997
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000,construction new jersey 3 directions park ave ...,40.7468,-73.9814
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027,cleared incident m14dbus 14th street,40.6701,-73.997
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000,construction q12bus alameda ave https co dov n...,43.6918,-79.4378


In [43]:
temp2.columns = ['original tweet', 'neutral score', 'polar score', 'trimmed tweet', 'latitude', 'longitude']
temp2

,original tweet,neutral score,polar score,trimmed tweet,latitude,longitude
0,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,cleared incident southern state park way wb ex...,40.792427,-73.827778
2,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,construction i84e eb exit 39a e 9 sb https co ...,40.714550,-74.007140
3,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,incident new jersey 181 directions cr 517 spaa...,40.714550,-74.007140
4,Incident on #Q15ABus from Roosevelt Ave: Main ...,1.000,0.0000,incident q15abus roosevelt ave main st rooseve...,40.678550,-73.588960
6,Incident on #Q12Bus from Roosevelt Ave: Main S...,1.000,0.0000,incident q12bus roosevelt ave main st roosevel...,40.678550,-73.588960
...,...,...,...,...,...,...
17781,Incident on #I87Northway SB from Exit 11; Roun...,1.000,0.0000,incident i87noh way sb exit 11 round lake road...,40.762081,-73.708859
17783,Cleared: Special Event on #7ThStreet EB from J...,0.559,0.7269,cleared special event 7th street eb jackson st...,40.720793,-74.000565
17785,Cleared: Incident on #M14ABus at 14th Street.,0.811,0.1027,cleared incident m14abus 14th street,40.670122,-73.997012
17786,Construction on #NJ3 Both directions from Park...,1.000,0.0000,construction new jersey 3 directions park ave ...,40.746757,-73.981437


In [46]:
temp2.to_csv('data_twitter/locat_6a.csv', mode='a')

In [45]:
temp2 = temp2.reset_index(drop=True)

freq = pd.DataFrame()
freq2 = temp2.groupby(["longitude", "latitude"]).size().reset_index(name="Frequency")

freq2 = freq2.sort_values('Frequency', ascending=False)

freq2.drop(freq2.head(1).index,inplace=True)
# most frequent value is the default value that returns location of 'new york'

freq2[['polar score', 'neutral score']] = temp2[['polar score', 'neutral score']]

freq2

,longitude,latitude,Frequency,polar score,neutral score
1083,-73.829990,40.714000,169,0.1027,0.865
473,-73.990360,40.692450,102,0.0000,1.000
974,-73.887080,40.676210,89,0.1027,0.851
906,-73.910857,40.776001,85,0.1027,0.851
237,-74.032775,40.773145,81,0.0000,1.000
...,...,...,...,...,...
447,-73.993910,40.736200,1,0.0000,1.000
448,-73.993786,40.702586,1,0.0000,1.000
957,-73.895559,40.674659,1,0.1027,0.896
955,-73.895965,40.736454,1,0.1027,0.924
